In [6]:
# Regular python libraries
from ipywidgets import interact
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

# Standar bokeh libraries
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import LinearAxis, Range1d  # Handle two y axis
from bokeh.models import CheckboxGroup, CustomJS
from bokeh.layouts import row, column, widgetbox
import warnings
warnings.filterwarnings('ignore')
output_notebook()

Loading BokehJS ...

In [7]:
# Read data (from file or DB)
df = pd.read_csv('acceleration.csv',#../Dataset/train/00001/acceleration.csv',# skiprows=5,
    sep=',')
#df.head()

In [8]:
acc_min = -8
acc_max = 8
ap_min = -110
ap_max = 0
dfa = df.copy()
len_before = len(dfa.index)
dfz = dfa[((dfa.x > acc_min) < acc_max)][((dfa.y > acc_min) < acc_max)][((dfa.z > acc_min) < acc_max)]\
[((dfa.Kitchen_AP < ap_max) & (dfa.Kitchen_AP > ap_min)) | dfa.Kitchen_AP.isnull()]\
[((dfa.Lounge_AP < ap_max) & (dfa.Lounge_AP > ap_min)) | dfa.Lounge_AP.isnull()]\
[((dfa.Upstairs_AP < ap_max) & (dfa.Upstairs_AP > ap_min)) | dfa.Upstairs_AP.isnull()]\
[((dfa.Study_AP < ap_max) & (dfa.Study_AP > ap_min)) | dfa.Study_AP.isnull()]\
[(dfa.t > 0) & (dfa.t < 30 * 60 * 20)]
len_after = len(dfz.index)
if len_before != len_after:
    print('There were outranged data')
else:
    print('Same!')


Same!


In [9]:
# Setting column 't' as timedeltaindex
dfa = dfz.copy()
dfa['t'] = pd.to_timedelta(dfa['t'],unit='s')
dfa = dfa.set_index(pd.TimedeltaIndex(dfa['t']))
del dfa['t']
dfa.head()

,x,y,z,Kitchen_AP,Lounge_AP,Upstairs_AP,Study_AP
00:00:00.017856,0.944,-0.280,0.152,-93.0,-95.0,-79.0,NaN
00:00:00.067904,0.944,-0.292,0.156,-93.0,-95.0,-79.0,NaN
00:00:00.117952,0.946,-0.286,0.156,-93.0,-95.0,-79.0,NaN
00:00:00.168000,0.942,-0.288,0.160,-93.0,-95.0,-79.0,NaN
00:00:00.217856,0.946,-0.286,0.158,-90.0,-89.0,-79.0,NaN


In [10]:
# Sampling function 
def sampling_data(timeunit, sampleps, from_time, to_time):
    freq = str(int(1000/sampleps))+'L'
    dfb = dfa.resample(freq).mean()
    dfc = dfb[dfb.index.slice_indexer(str(int(from_time))+timeunit,str(to_time-0.001)+timeunit,1)]
    return dfc

In [11]:
# Some initial values to plot
sampleps = 1
timeunit = 'M'
from_time = 0
to_time = 10
dfc = sampling_data(timeunit, sampleps, from_time, to_time)
#dfc.head()

In [12]:
acc_dim = ['x','y','z']
dim_color = {'x':"#008744",'y':"#0057e7",'z':"#d62d20"}
f = figure(title="Acceleration Datafile", plot_height=400, plot_width=500, x_axis_type="datetime")
f.extra_y_ranges = {"ACC": Range1d(start=-4, end=2)} # Configuring a new Y range
f.add_layout(LinearAxis(y_range_name="ACC", axis_label='Acc Range'), 'right')

In [13]:
# XYZ Lines
acc_x = f.line(dfc.index,dfc['x'], color=dim_color['x'], line_width=1, y_range_name='ACC', legend='X') 
acc_y = f.line(dfc.index,dfc['y'], color=dim_color['y'], line_width=1, y_range_name='ACC', legend='Y') 
acc_z = f.line(dfc.index,dfc['z'], color=dim_color['z'], line_width=1, y_range_name='ACC', legend='Z') 

In [14]:
# AP Data
aps = ['Kitchen_AP', 'Lounge_AP', 'Upstairs_AP', 'Study_AP']
ap_color = {'Kitchen_AP':"#008744",'Lounge_AP':"#0057e7",'Upstairs_AP':"#d62d20", 'Study_AP':"#ffa700"}
acc_Kitchen_AP = f.circle(dfc.index,dfc['Kitchen_AP'], size=4, color=ap_color['Kitchen_AP'], alpha=0.2, legend='Kitchen_AP')
acc_Lounge_AP = f.circle(dfc.index,dfc['Lounge_AP'],size=4 , color=ap_color['Lounge_AP'], alpha=0.2, legend='Lounge_AP')
acc_Upstairs_AP = f.circle(dfc.index,dfc['Upstairs_AP'], size=4, color=ap_color['Upstairs_AP'], alpha=0.2, legend='Upstairs_AP')
acc_Study_AP = f.circle(dfc.index,dfc['Study_AP'], size=4, color=ap_color['Study_AP'], alpha=0.2, legend='Study_AP')

In [15]:
# I'm pretty sure there should be a better way of doing it...
def update(timeunit = 'M', sampleps = 1, from_time=0, to_time=10):
    dfc = sampling_data(timeunit, sampleps, from_time, to_time)
    acc_x.data_source.data['x']=dfc.index
    acc_x.data_source.data['y']=dfc['x']
    acc_y.data_source.data['x']=dfc.index
    acc_y.data_source.data['y']=dfc['y']
    acc_z.data_source.data['x']=dfc.index
    acc_z.data_source.data['y']=dfc['z']
    acc_Kitchen_AP.data_source.data['x']=dfc.index
    acc_Kitchen_AP.data_source.data['y']=dfc['Kitchen_AP']
    acc_Lounge_AP.data_source.data['x']=dfc.index
    acc_Lounge_AP.data_source.data['y']=dfc['Lounge_AP']
    acc_Upstairs_AP.data_source.data['x']=dfc.index
    acc_Upstairs_AP.data_source.data['y']=dfc['Upstairs_AP']
    acc_Study_AP.data_source.data['x']=dfc.index
    acc_Study_AP.data_source.data['y']=dfc['Study_AP']
    push_notebook()

In [16]:
# Fancy checkbox that still doesn't work quite well
code = """
    if (0 in object.active) {
        line1.visible = true
    } else {
        line1.visible = false
    }
    if (1 in object.active) {
        line2.visible = true
    } else {
        line2.visible = false
    }
    if (2 in object.active) {
        line3.visible = true
    } else {
        line3.visible = false
    }
"""
callback = CustomJS(code=code, args={})
checkbox = CheckboxGroup(labels=["X", "Y", "Z"], active=[0, 1, 2], callback=callback, inline=True, width=150)
callback.args = dict(line1=acc_x, line2=acc_y, line3=acc_z, object=checkbox)#l1=l1, l2=l2, checkbox=checkbox)

In [17]:
show(column(widgetbox(checkbox),f), notebook_handle=True)

In [18]:
interact(update, timeunit = ["M","S"],sampleps = (1,20), from_time = (0,29), to_time = (1,30))

<function __main__.update>